In [ ]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division
%matplotlib inline 

In [ ]:
DIR = '/home/lindb/teakettle/data/moran/dependencies'
if not op.exists(DIR):
    os.makedirs(DIR)

In [ ]:
610*610

In [ ]:
DIR = '/home/lindb/teakettle/data/moran/amat_files'
if not op.exists(DIR):
    os.makedirs(DIR)

for k in np.arange(610): # nadult
    i = k+1 #R doesn't have zeroth elements (lame)
    if i % 100 == True:
        print i
    for w in np.arange(610):
        j = w+1
        text = '''
source('/home/lindb/imports.R')

#load dependencies
dname = '/home/lindb/teakettle/data/moran/dependencies/'
files = c()
for (f in list.files(dname)){
    if (endsWith(f,'RDS'))
    {
        sprintf ("loading %%s file",f)
        fname = paste(dname,f,sep="")
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(fname))    
    }
}

amat <- numeric(0)
for (k in 1:noff)
{
    a <- matrix(0,(nadult+1),(nadult+1)) 
    amat <- append(amat,list(a))
}
    
i <- %s                                                                           #########################
j <- %s                                                                           #########################
cat(sprintf ("i=%%s",i))                                                           
cat(sprintf ("j=%%s",j))                                                           



print ("starting loop")
if(i==j & i<(nadult+1)) #this is the diagonal, the mother and father aren't likely to be the same tree
{
    unimportant = 1+1 # amat is already 0 on the diagonal, save some time by not looping
#         for (k in 1:noff)
#         {
#             amat[[k]][i,j] <- 0   
#         }
}
if(i==j & i==(nadult+1)) #can have two out-of-plot parents
{ 
    nmat <- numeric(0)
    pg1 <- pgout 
    pg1.b <- pgout 
    pg2 <- pgout 
    pg2.b <- pgout
    print ("starting loci loop")
    for(la in 1:nloci) 
    {#Calculate probability of offspring genotype given parental genotypes using global allele frequencies
        #at the la'th locus
        print (la)
        nm <- nmat.func(pg1,pg2,pg1.b,pg2.b,la) #returns a matrix with ncol = nrow = nallele @ la'th locus
        nmat <- append(nmat,list(nm))
    }
    print ("starting offspring loop")
    for(k in 1:noff)
    {
        print (k)
        pq <- 1 #incorrectly overwriting pq is bad, mkay
        og <- rbind(goff[[1]][k,],goff[[2]][k,]) #get the offspring's observed first and second alleles
        og.b <- rbind(goff.r[[1]][k,],goff.r[[2]][k,]) #get any offspring's observed regenotyping results
        print ("starting loci loop")
        for(la in 1:nloci)
        {
            #print (la)
            #om = probability of observing the given offspring genotype given all possible true genotypes
            om <- omat.func(og,og.b,la) 
            
            #pq = probability of observing the given offspring genotype given the observed parental genotypes
            pq <- pq*sum(nmat[[la]]*om)
            
            #if the posited parents are not the parents, no use in doing futher calculations @ other loci
            if(pq==0) 
            {
                break 
            }
        }
        print ("writing amat")
        amat[[k]][i,j] <- pq #pq same as above, for the k'th offspring for the i'th and j'th parent
        amat[[k]][j,i] <- pq #since i==j, it's ok to flip over diagonal
    }# end offspring loop
}# end two-out-of-plot loop
if(i!=j) #if not on the diagonal
{ #all other combos... 
    nmat <- numeric(0)
    if(i<(nadult+1)) #if posited mother one of those who were sampled
    {
        #gadult# == nrow=nadult ncol=nloci
        pg1 <- rbind(gadult[[1]][i,],gadult[[2]][i,])  #get first and second alleles for all loci
        pg1.b <- rbind(gadult.r[[1]][i,],gadult.r[[2]][i,]) #get first and second alleles for regenotyping
    } 
    if(j<(nadult+1)) #if posited father one of those who were sampled
    {
        pg2 <- rbind(gadultF[[1]][j,],gadultF[[2]][j,])
        pg2.b <- rbind(gadultF.r[[1]][j,],gadultF.r[[2]][j,]) 
    }
    if(i==(nadult+1)) #if posited mother is not one of those who were sampled
    { 
        pg1 <- pgout 
        pg1.b <- pgout
    } 
    if(j==(nadult+1)) #if posited father is not one of those who were sampled
    {
        pg2 <- pgout #put in all zeros
        pg2.b <- pgout #put in all zeros
    }
    print ("starting loci loop")
    for(la in 1:nloci)
    {#Calculate probability of offspring genotype given parental genotypes using global allele frequencies
        #at the la'th locus
        print (la)
        nm <- nmat.func(pg1,pg2,pg1.b,pg2.b,la) 
        nmat <- append(nmat,list(nm))
    }
    print ("starting offspring loop")
    for(k in 1:noff)
    { #pre-calc nmat at all loci for parent pair
        print(k)
        pq <- 1 #incorrectly overwriting is still bad, mkay
        og <- rbind(goff[[1]][k,],goff[[2]][k,]) #get the k'th offspring's 1st and 2nd alleles @ all loci
        og.b <- rbind(goff.r[[1]][k,],goff.r[[2]][k,])
        print ("starting loci loop")
        for(la in 1:nloci)
        {
            print (la)
            #om = probability of observing the given offspring genotype given all possible true genotypes
            om <- omat.func(og,og.b,la)
            #pq = probability of observing the given offspring genotype given the observed parental genotypes
            pq <- pq*sum(nmat[[la]]*om)
            if(pq==0) 
            {
            #if the posited parents are not the parents, no use in doing futher calculations @ other loci
                break 
            }
        }
        amat[[k]][i,j] <- pq 
        #amat[[k]][j,i] <- pq # I'm pretty sure I don't want to do this, since I'm specifying mother as i 
                             # and father as j
    }# end offspring loop
}#end "all other combos" loop


print ("writing RDS files")
DIR = '/home/lindb/teakettle/data/amat_outfiles/'
amatfname = paste(DIR,'amat_%s_%s.RDS',sep="")                                   ######################### 2x
nmatfname = paste(DIR,'nmat_%s_%s.RDS',sep="")                                   ######################### 2x
sprintf("writing %%s",amatfname)
saveRDS(amat,amatfname)
sprintf("writing %%s",nmatfname)
saveRDS(nmat,nmatfname)

''' % (i,
       j,
       str(i).zfill(3),
       str(j).zfill(3),
       str(i).zfill(3),
       str(j).zfill(3)
          )
        DIR = '/home/lindb/teakettle/data/moran/amat_files2'
        if not op.exists(DIR):
            os.makedirs(DIR)
        filE = op.join(DIR,"amat_%s_%s.R" % (str(i).zfill(3),str(j).zfill(3)))
        with open(filE,'wb') as o:
            o.write("%s" % text)

# dont forget about amat[[k]] <- amat[[k]]/sum(amat[[k]])
I'm pretty sure I was talking about when I did this the first time without distinguishing 
mother as i and father as j and didn't reflect across diagonal?

In [ ]:
DIR = '/home/lindb/teakettle/data/amat_outfiles/'
if not op.exists(DIR):
    os.makedirs(DIR)

### remake r files for j > i, difference is in how to store amat[[k]][i,j]

this time amat[[k]] is not a matrix, it's a single value

I have to do this because I'm considering the father as j and the mother as i (see 03_parentage_R.ipynb for functions)

In [ ]:
x = pd.DataFrame(0,index=xrange(1,6),columns=xrange(1,6))
for q in range(5):
    i = q+1
    for w in range(5):
        j = w+1
        if i < j:
            x.loc[i,j] = "done"
x

In [ ]:
#make sure I won't waste time calling then skipping unneeded values
for q in range(5):
    i = q+1
    for w in range(i-1):
        j = w+1
        x.loc[i,j] = 'needed'
        if not j < i:
            print "hmm"
x

In [ ]:
#where to store remake RDS files
DIR = '/home/lindb/teakettle/data/amat_outfiles/remake'
if not op.exists(DIR):
    os.makedirs(DIR)

#where to put R files
DIR = '/home/lindb/teakettle/data/moran/amat_files2/remake'
if not op.exists(DIR):
    os.makedirs(DIR)
    
for q in range(558):
    i = q+1
    for w in range(i-1):
        j = w+1
        text = '''
source('/home/lindb/imports.R')

#load dependencies
dname = '/home/lindb/teakettle/data/moran/dependencies/'
files = c()
for (f in list.files(dname)){
    if (endsWith(f,'RDS'))
    {
        sprintf ("loading %%s file",f)
        fname = paste(dname,f,sep="")
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(fname))    
    }
}

#here is where the change is
#will take less time to make, less time to saveRDS
amat <- numeric(0)
for (k in 1:noff)
{
    #a <- matrix(0,(nadult+1),(nadult+1)) 
    a <- 0 #will be replaced by a single value
    amat <- append(amat,list(a))
}
    
i <- as.numeric(%s)
j <- as.numeric(%s)
cat(sprintf ("i=%%s",i))
cat(sprintf ("j=%%s",j))



print ("starting loop")
#no need to waste time with diagonal
#if(i==j & i<(nadult+1)) #this is the diagonal, mother and father can't have been the same tree
#{
#    unimportant = 1+1 # amat is already 0 on the diagonal, save some time by not looping
#    for (k in 1:noff)
#    {
#        amat[[k]] <- 0   
#    }
#}
if(i==j & i==(nadult+1)) #can have two out-of-plot parents
{ 
    nmat <- numeric(0)
    pg1 <- pgout 
    pg1.b <- pgout 
    pg2 <- pgout 
    pg2.b <- pgout
    print ("starting loci loop")
    for(la in 1:nloci) 
    {#Calculate probability of offspring genotype given parental genotypes using global allele frequencies
        #at the la'th locus
        print (la)
        nm <- nmat.func(pg1,pg2,pg1.b,pg2.b,la) #returns a matrix with ncol = nrow = nallele @ la'th locus
        nmat <- append(nmat,list(nm))
    }
    print ("starting offspring loop")
    for(k in 1:noff)
    {
        #print (k)
        pq <- 1 #incorrectly overwriting pq is bad, mkay
        og <- rbind(goff[[1]][k,],goff[[2]][k,]) #get the offspring's observed first and second alleles
        og.b <- rbind(goff.r[[1]][k,],goff.r[[2]][k,]) #get any offspring's observed regenotyping results
        #print ("starting loci loop")
        for(la in 1:nloci)
        {
            #print (la)
            #om = probability of observing the given offspring genotype given all possible true genotypes
            om <- omat.func(og,og.b,la) 
        #pq = probability of observing the given offspring genotype given the observed parental genotypes
            pq <- pq*sum(nmat[[la]]*om)
            #if the posited parents are not the parents, no use in doing futher calculations @ other loci
            if(pq==0) 
            {
                break 
            }
        }
        print ("writing amat")
        #amat[[k]][i,j] <- pq #pq same as above, for the k'th offspring for the i'th and j'th parent
        #amat[[k]][j,i] <- pq 
        amat[[k]] <- pq
    }# end offspring loop
}# end two-out-of-plot loop
if(i!=j) #if not on the diagonal
{ #all other combos... 
    nmat <- numeric(0)
    if(i<(nadult+1)) #if posited mother one of those who were sampled
    {
        #gadult# == nrow=nadult ncol=nloci
        pg1 <- rbind(gadult[[1]][i,],gadult[[2]][i,])  #get first and second alleles for all loci
        pg1.b <- rbind(gadult.r[[1]][i,],gadult.r[[2]][i,]) #get first and second alleles for regenotyping
    } 
    if(j<(nadult+1)) #if posited father one of those who were sampled
    {
        pg2 <- rbind(gadultF[[1]][j,],gadultF[[2]][j,])
        pg2.b <- rbind(gadultF.r[[1]][j,],gadultF.r[[2]][j,]) 
    }
    if(i==(nadult+1)) #if posited mother is not one of those who were sampled
    { 
        pg1 <- pgout   #put in all zeros
        pg1.b <- pgout #put in all zeros
    } 
    if(j==(nadult+1)) #if posited father is not one of those who were sampled
    {
        pg2 <- pgout   #put in all zeros
        pg2.b <- pgout #put in all zeros
    }
    print ("starting loci loop")
    for(la in 1:nloci)
    {#Calculate probability of offspring genotype given parental genotypes using global allele frequencies
        #at the la'th locus
        print (la)
        nm <- nmat.func(pg1,pg2,pg1.b,pg2.b,la) 
        nmat <- append(nmat,list(nm))
    }
    print ("starting offspring loop")
    for(k in 1:noff)
    { #pre-calc nmat at all loci for parent pair
        #print(k)
        pq <- 1 #incorrectly overwriting is still bad, mkay
        og <- rbind(goff[[1]][k,],goff[[2]][k,]) #get the k'th offspring's 1st and 2nd alleles @ all loci
        og.b <- rbind(goff.r[[1]][k,],goff.r[[2]][k,])
        #print ("starting loci loop")
        for(la in 1:nloci)
        {
            #print (la)
            #om = probability of observing the given offspring genotype given all possible true genotypes
            om <- omat.func(og,og.b,la)
            #pq = probability of observing the given offspring genotype given the observed parental genotypes
            pq <- pq*sum(nmat[[la]]*om)
            if(pq==0) 
            {
            #if the posited parents are not the parents, no use in doing futher calculations @ other loci
                break 
            }
        }
        #amat[[k]][i,j] <- pq 
        #amat[[k]][j,i] <- pq 
        amat[[k]] <- pq
    }# end offspring loop
}#end "all other combos" loop


print ("writing RDS files")
DIR = '/home/lindb/teakettle/data/amat_outfiles/remake/'
amatfname = paste(DIR,'amat_%s_%s.RDS',sep="")
#nmatfname = paste(DIR,'nmat_%s_%s.RDS',sep="") #not used later on so no need to save

#sprintf("writing %%s",amatfname)
saveRDS(amat,amatfname)

#sprintf("writing %%s",nmatfname)
#saveRDS(nmat,nmatfname)

''' % (i,
       j,
       str(i).zfill(3),
       str(j).zfill(3),
       str(i).zfill(3),
       str(j).zfill(3)
      )
        #DIR = '/home/lindb/teakettle/data/moran/amat_files2/remake'
        filE = op.join(DIR,"amat_%s_%s.R" % (str(i).zfill(3),str(j).zfill(3)))
        with open(filE,'wb') as o:
            o.write("%s" % text)

In [ ]:
len(ls('/home/lindb/teakettle/data/moran/amat_files2/remake'))

# make sh files to run R files

In [ ]:
#DIR = '/home/lindb/teakettle/data/moran/amat_files2/'
DIR = '/home/lindb/teakettle/data/moran/amat_files2/remake'
rfiles = fs(DIR)
luni(rfiles) #nadult choose 2

In [ ]:
rfiles[0]

In [ ]:
#realized I only need to run files where i < j (+ i=nadult j=nadult) #this is incorrect since I'm treating j as father
    #(bc only doing lowertri, reflecting across diagonal in Rscript, diag == 0)
todo = []
for q in range(558):
    i = q+1
    for w in range(558):
        j = w+1
        if i < j:
            filE = '/home/lindb/teakettle/data/moran/amat_files2/amat_%s_%s.R' % (str(i).zfill(3),str(j).zfill(3))
            if op.exists(filE):
                if not op.exists('/home/lindb/teakettle/data/amat_outfiles/amat_%s_%s.RDS' % (str(i).zfill(3),str(j).zfill(3))):
                    todo.append(filE)
            else:
                print filE
len(todo)

In [ ]:
#check len(todo)
count = 0
for i in range(558):
    for j in range(i):
        count += 1
count

In [ ]:
# how many r files per sh file?
math.ceil(len(todo)/197)

In [ ]:
# how many r files per sh file?
math.ceil(len(rfiles)/198)

In [ ]:
#where to put the sh files
#DIR = '/home/lindb/teakettle/data/moran/amat_runfiles/'
##DIR = '/home/lindb/teakettle/data/moran/amat_runfiles2/' #trying to run more jobs than previously was able
DIR = '/home/lindb/teakettle/data/moran/amat_runfiles2/remake'
if not op.exists(DIR):
    os.makedirs(DIR)

fcount = 0
shcount = 0
rcount = 0
##for f in sorted(todo):
for f in sorted(rfiles):
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N make%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s

''' % (str(shcount).zfill(3),
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount += 1
    rcount += 1
    
    #if rcount == 525 or fcount == len(todo):
    if rcount == 785 or fcount == len(rfiles):
        filE = op.join(DIR,'amat_%s.sh' % str(shcount).zfill(3))
        with open(filE,'wb') as o:
            o.write("%s" % text)
        
        rcount = 0
        shcount += 1

In [ ]:
#DIR = '/home/lindb/teakettle/data/moran/amat_runfiles/'
##DIR = '/home/lindb/teakettle/data/moran/amat_runfiles2/'
DIR = '/home/lindb/teakettle/data/moran/amat_runfiles2/remake'
cd(DIR) #so .o files don't end up some place weird
for f in fs(DIR):
    !qsub $f

## how much longer is this going to take? (still have to run gibbs sampler after this)

In [ ]:
#DIR = '/home/lindb/teakettle/data/amat_outfiles/'
DIR = '/home/lindb/teakettle/data/amat_outfiles/remake/'
n = 155403-11982 #nadult choose 2 - some files I had done before hijacking lab mates queue slots muahahahaha
#oldrates = rates
rates = []

In [ ]:
#plus minus is probably incorrect
def getrate(trec,tfirst,xrec,xfirst):
    diff = trec - tfirst
    days = (diff.seconds/86400) + diff.days
    hourly = round((xrec - xfirst)/(days*24),2)
    return(hourly)
    
def trem():
    if 'tnow' in globals():
        tpast = globals()['tnow']
        globals()['tpast'] = tpast
    if 'xnow' in globals():
        xpast = globals()['xnow']
        globals()['xpast'] = xpast
    tnow = dt.now()
    globals()['tnow'] = tnow
    xnow = len([f for f in ls(DIR) if 'amat' in f])
    globals()['xnow'] = xnow
    
    assert tnow > t0
    assert xnow > x0
    
    hourly = getrate(tnow,t0,xnow,x0)
    rates.append(hourly)
    recrate = getrate(tnow,tpast,xnow,xpast)
    std = np.std(rates)
    
    t_rem    = (n-xnow)/np.mean([x*24 for x in rates])
    t_remhi = (n-xnow)/(interval(0.68,
                                 loc=np.mean([x*24 for x in rates]),
                                 scale=np.std(rates)/np.sqrt(len(rates)))[1])
    t_remerr = abs(t_remhi - t_rem)*24*60
    
    splits = str(t_rem).split(".")
    
    if len(splits) > 1:
        t_rem = "measured @ %s\n\
%s days %s hours +/- %sminutes\n\
@ mean %s per hour\n\
@ current rate %s per hour\n\
mean rate %s\n\
" % (tnow,
     splits[0],
     round(float(".%s" % splits[1])*24,
           2),
     '%.2E' % Decimal(t_remerr),
     hourly,
     recrate,
     round(np.mean(rates),
          2)) #fix


    print(t_rem)

In [ ]:
#plus minus is probably incorrect
def getrate(trec,tfirst,xrec,xfirst):
    diff = trec - tfirst
    days = (diff.seconds/86400) + diff.days
    hourly = round((xrec - xfirst)/(days*24),2)
    return(hourly)
    
def trem():
    if 'tnow' in globals():
        tpast = globals()['tnow']
        globals()['tpast'] = tpast
    if 'xnow' in globals():
        xpast = globals()['xnow']
        globals()['xpast'] = xpast
    tnow = dt.now()
    globals()['tnow'] = tnow
    xnow = len([f for f in ls(DIR) if 'amat' in f])
    globals()['xnow'] = xnow
    
    assert tnow > t0
    assert xnow > x0
    
    hourly = getrate(tnow,t0,xnow,x0)
    rates.append(hourly)
    #recrate = getrate(tnow,tpast,xnow,xpast)
    std = np.std(rates)
    
    t_rem    = (n-xnow)/np.mean([x*24 for x in rates])
    t_remhi = (n-xnow)/(interval(0.68,
                                 loc=np.mean([x*24 for x in rates]),
                                 scale=np.std(rates)/np.sqrt(len(rates)))[1])
    t_remerr = abs(t_remhi - t_rem)*24*60
    
    splits = str(t_rem).split(".")
    
    if len(splits) > 1:
        t_rem = "measured @ %s\n\
%s days %s hours +/- %sminutes\n\
@ mean %s per hour\n\
mean rate %s\n\
" % (tnow,
     splits[0],
     round(float(".%s" % splits[1])*24,
           2),
     '%.2E' % Decimal(t_remerr),
     hourly,
     round(np.mean(rates),
          2)) #fix


    print(t_rem)

In [ ]:
#t0 - I was able to increase number of sh files a little while after qsubbing a lesser number of sh files
# t0 = dt(2017, 6, 2, 10, 15, 0, 0)
# x0 = 65374 #len([f for f in ls(DIR) if 'amat' in f]) #10:15 6/2/17
t0 = dt(2017, 6, 10, 22, 11, 58, 235437)
x0 = 856 #len([f for f in ls(DIR) if 'amat' in f]) #22:10 6/10/17

In [ ]:
files = ls('/home/lindb/teakettle/data/amat_outfiles/remake/')
files[0]

In [ ]:
'amat_358_050.RDS'[:-2]

In [ ]:
DIR = '/gpfs_fs/home/eckertlab/trevor/brandon/home/lindb/teakettle/data/amat_outfiles/remake/'
trem()

plt.hist(rates)

In [ ]:
DIR = '/gpfs_fs/home/eckertlab/trevor/brandon/home/lindb/teakettle/data/amat_outfiles/remake/'
trem()

plt.hist(rates)

In [ ]:
DIR = '/gpfs_fs/home/eckertlab/trevor/brandon/home/lindb/teakettle/data/amat_outfiles/remake/'
trem()

plt.hist(rates)

In [ ]:
DIR = '/gpfs_fs/home/eckertlab/trevor/brandon/home/lindb/teakettle/data/amat_outfiles/remake/'
trem()

plt.hist(rates)

In [ ]:
DIR = '/gpfs_fs/home/eckertlab/trevor/brandon/home/lindb/teakettle/data/amat_outfiles/remake/'
trem()

plt.hist(rates)

In [ ]:
xnow

In [ ]:
tnow - tpast

In [ ]:
(xnow-xpast)/((tnow - tpast).seconds*86400)

In [ ]:
tn = dt.now()
xn = len(fs('/gpfs_fs/home/eckertlab/trevor/brandon/home/lindb/teakettle/data/amat_outfiles/remake'))
xn

In [ ]:
t0

# will take forever to load each RDS in loop, make smaller sizes

### next time around I'll just save the files as follows:

In [ ]:
os.getcwd()

In [ ]:
sDIR = '/home/lindb/teakettle/data/small/small_amat_outfiles'
if not op.exists(sDIR):
    os.makedirs(sDIR)
    
rDIR = '/home/lindb/teakettle/data/small/r_runfiles'
if not op.exists(rDIR):
    os.makedirs(rDIR)
    
DIR = '/home/lindb/teakettle/data/amat_outfiles'
files = [op.join(DIR,f) for f in ls(DIR) if 'amat' in f]

for f in files:
    i,j = op.basename(f).split(".")[0].split("_")[1:]
    text ='''
source('/home/lindb/imports.R')

amat = readRDS('%s')

i = as.numeric(%s)
j = as.numeric(%s)

sm = numeric(0)
for (k in 1:len(amat)){
    sm <- append(sm,list(amat[[k]][i,j]))
}

F = '%s'
saveRDS(sm,F)


''' % (f,
       i,
       j,
       op.join(sDIR,"%s_%s.RDS" % (i,j)),
      )
    filE = op.join(rDIR,"%s_%s.R" % (i,j))
    with open(filE,'w') as o:
        o.write("%s" % text)

In [ ]:
files = fs('/home/lindb/teakettle/data/small/r_runfiles')
fcount = 0
shcount = 0
rcount = 0
DIR = '/home/lindb/teakettle/data/small/sh_runfiles'
if not op.exists(DIR):
    os.makedirs(DIR)
for f in sorted(files):
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N small%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s

''' % (str(shcount).zfill(3),
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount += 1
    rcount += 1
    
    if rcount == 785 or fcount == len(files):
        filE = op.join(DIR,'small_amat_%s.sh' % str(shcount).zfill(3))
        with open(filE,'wb') as o:
            o.write("%s" % text)
        
        rcount = 0
        shcount += 1

In [ ]:
#/gpfs_fs/home/lindb/teakettle/data/moran/amat_runfiles2

In [ ]:
DIR = '/home/lindb/teakettle/data/small/sh_runfiles'
shfiles = fs(DIR)
cd(DIR) #want .o files in DIR
for f in shfiles:
    !qsub $f

In [ ]:
DIR = '/home/lindb/teakettle/data/small/small_amat_outfiles/'
len(ls(DIR))

In [ ]:
files = fs('/home/lindb/teakettle/data/small/small_amat_outfiles')
files[0]

In [ ]:
todo = []
for q in range(558):
    i = q+1
    for w in range(558):
        j = w+1
        if i < j:
            filE = '/home/lindb/teakettle/data/small/small_amat_outfiles/%s_%s.RDS' % (str(i).zfill(3),
                                                                                       str(j).zfill(3)
                                                                                      )
            if not op.exists(filE):
                todo.append(filE)
len(todo)

In [ ]:
count = 0
for t in todo:
    nums = op.basename(t).split(".")[0]
    filE = '/home/lindb/teakettle/data/amat_outfiles/amat_%s.RDS' % nums
    if op.exists(filE):
        print filE
        count +=1
count

In [ ]:
sDIR = '/home/lindb/teakettle/data/small/small_amat_outfiles'
if not op.exists(sDIR):
    os.makedirs(sDIR)
    
rDIR = '/home/lindb/teakettle/data/small/r_runfiles/left'
if not op.exists(rDIR):
    os.makedirs(rDIR)
    
DIR = '/home/lindb/teakettle/data/amat_outfiles'
files = [op.join(DIR,f) for f in ls(DIR) if 'amat' in f]

for t in todo:
    nums = op.basename(t).split(".")[0]
    f = '/home/lindb/teakettle/data/amat_outfiles/amat_%s.RDS' % nums
    assert op.exists(f)
    i,j = nums.split("_")
    text ='''
source('/home/lindb/imports.R')

amat = readRDS('%s')

i = as.numeric(%s)
j = as.numeric(%s)

sm = numeric(0)
for (k in 1:len(amat)){
    sm <- append(sm,list(amat[[k]][i,j]))
}

F = '%s'
saveRDS(sm,F)


''' % (f,
       i,
       j,
       op.join(sDIR,"%s_%s.RDS" % (i,j)),
      )
    filE = op.join(rDIR,"%s_%s.R" % (i,j))
    with open(filE,'w') as o:
        o.write("%s" % text)

In [ ]:
files = [f for f in fs('/home/lindb/teakettle/data/small/r_runfiles/left') if 'sh' not in f]
count = 0
for f in sorted(files):
    if count == 0:
        text = '''#!/bin/bash
#$ -N oops
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s
''' % (op.dirname(f),
       f
      )
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f
      )
        text = text + newtext
    count += 1
filE = '/home/lindb/teakettle/data/small/r_runfiles/left/left.sh'
with open(filE,'w') as o:
    o.write("%s" % text)

# combine small amat files into one amat file

In [ ]:
DIR = '/home/lindb/teakettle/data/small/combo'
if not op.exists(DIR):
    os.makedirs(DIR)
text = '''
source('~/imports.R')

#load dependencies
dname = '/home/lindb/teakettle/data/moran/dependencies/'
files = c()
for (f in list.files(dname)){
    if (endsWith(f,'RDS'))
    {
        cat(sprintf ("loading %s file\n",f))
        fname = paste(dname,f,sep="")
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(fname))    
    }
}

#make blank amat object
amat <- numeric(0)
for (k in 1:noff)
{
	a <- matrix(0,(nadult+1),(nadult+1)) 
	amat <- append(amat,list(a))
}

files = list.files('/home/lindb/teakettle/data/small/small_amat_outfiles/',full.name=T)
print(len(files))
count = 1
print(Sys.time())
for (f in files)
{
    if(count %% 1000 == 0){
        print(count)
    }
    vals <- readRDS(f)
    nums <- strsplit(basename(f),".RDS")[[1]]
    nums <- strsplit(nums,"_")[[1]]
    i <- as.numeric(nums[1])
    j <- as.numeric(nums[2])
    for (k in 1:noff)
    {
        amat[[k]][i,j] <- vals[[k]]
    }
    count = count + 1
}

filE = '/home/lindb/teakettle/data/small/combo/amat_upper_tri.RDS'
saveRDS(amat,filE)
'''
filE = op.join(DIR,'combine_upper_tri.R')
with open(filE,'w') as o:
    o.write("%s" % text)
    
#ran in tmux

# add in lower tri next round

# sum amat file 

### run this in R terminal
```R
source('/home/lindb/imports.R')

noff <- readRDS('/home/lindb/teakettle/data/moran/dependencies/noff.RDS')

amat <- readRDS('/home/lindb/teakettle/data/small/combo/amat_upper_tri.RDS')

for (k in 1:noff){
    amat[[k]] <- amat[[k]]/sum(amat[[k]])
}

saveRDS(amat,'/home/lindb/teakettle/data/small/combo/amat_upper_tri_summed.RDS')
```

# test run the Gibbs sampler while other data is finishing

In [ ]:
#too slow, optimized below with parallel engines
text ='''
source('/home/lindb/imports.R')

#load dependencies
dname = '/home/lindb/teakettle/data/moran/dependencies/'
files = c()
for (f in list.files(dname)){
    if (endsWith(f,'RDS'))
    {
        sprintf ("loading %%s file",f)
        fname = paste(dname,f,sep="")
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(fname))    
    }
}

pld <- as.numeric(pld)

amat <- readRDS('/home/lindb/teakettle/data/small/combo/amat_upper_tri_summed.RDS')
###### just this time - fix when I have lower tri
for (k in 1:noff){
  amat[[k]][lower.tri(amat[[k]])] <- t(amat[[k]])[lower.tri(amat[[k]])]
}

###############################################################
################## Step 5: the Gibbs Sampler ##################
###############################################################
#number of Gibbs steps
ncyc <- 50000
#Starting values for seed and pollen parameters
usgibbs <- us <- 150
upgibbs <- up <- 500  

###Which parent pairs will you consider? 
##Can consider all with a >0 probability of producing observed 
#offspring genotype (as below) or can set more stringent standard.
ppmat <- numeric(0)
for(k in 1:noff)
{
    print(k)
	b <- matrix(0,(nadult+1),(nadult+1))
    b[which(amat[[k]] > 0)] <- 1 #way faster than this loop below
#	for(i in 1:(nadult+1))
#	{
#		for(j in 1:(nadult+1))
#		{ 
#			##amat_k[i,j] represents the probability of obtaining the observed genotype of 
#			#offspring k given the pedigree, P_k = (i,j), relative to all possible parent combinations
#			if(amat[[k]][i,j]>0)
#			{
#				b[i,j]<-1
#			}
#		}
#	}
	#ppmat is a list of matrices of len(noff), each element a matrix of 0s and 1s. 1s = Amat_k[i,j]>0
	ppmat <- append(ppmat,list(b)) 
}

#Initial parentage values – start with parents that have high probability of producing observed offspring genotypes.
Ped.eg <- matrix(0,noff,2)
print("IDing first parents")
for (k in 1:noff)
{ #translates probabilities into a vector, upper triangle first, by column
	theta <- c(amat[[k]][upper.tri(amat[[k]])],amat[[k]][lower.tri(amat[[k]])],amat[[k]][(nadult+1),(nadult+1)])
	y     <- rmultinom(1,1,theta) 
	# ly = twice the total number of parent pair combinations where i!=j (plus one (amat_k[bottom right cell]), minus one)
	ly    <- length(y)-1 
	ul    <- .5*ly; #total number of parent pair combinations where i!=j
	ll    <- ul+1 #total number of parent pair combinations where i!=j plus one
	v     <- matrix(0,(nadult+1),(nadult+1)) 
	v[upper.tri(v)] <- y[c(1:ul)] #fills in the first half of multinomially distributed random number (parent pair combos)
	v[lower.tri(v)] <- y[c(ll:ly)] #fills in the second half of multinomially distributed random number (parent pair combos)
	#next add in the multinomially distributed random number corresponding to two parents outside the plot
	v[(nadult+1),(nadult+1)] <- y[length(y)] 
	Ped.eg[k,1] <- which(apply(v,1,sum)==1) #figure out the i'th coordinate(s) for the parent pair(s) within the matrix
	Ped.eg[k,2] <- which(apply(v,2,sum)==1) #figure out the j'th coordinate(s) for the parent pair(s) within the matrix
}

## to hold parentage estimates
ped.gibbs <- numeric(0)
ped.gibbs <- append(ped.gibbs,list(matrix(0,noff,2)))
ped.gibbs[[1]] <- Ped.eg
#Begin Gibbs Sampler:
#set all NA probs to ~0 -
print("starting Gibbs")
for (d in 1:ncyc)
{
  print(d)
  ped.gibbs <- append(ped.gibbs,list(matrix(0,noff,2)))
  ped.new   <- matrix(0,noff,2)
  tnow      <- rep(0,noff); 
  tnow.s    <- rep(0,noff) 
  tnew      <- rep(0,noff); 
  tnew.s    <- rep(0,noff)
  if(d==1)
  { #just the first time
    #Probability of seed dispersal from sampled adult to location of each seedling:
    dprob <- dprob.func(us) #seedling shape parameter, us, defined around line 1042
    dprob[which(is.na(dprob))] <- 1e-100
    #Probability of pollen dispersal between sampled adults:
    pprob <- pprob.func(up) #pollen shape parameter, up, defined around line 1043
    pprob[which(is.na(pprob))] <- 1e-100 
    #Expected seed arriving at seedling plots from outside plot:
    osd   <- out.sd.func(us) ##osd[i,j] is the total probability of a seed 
    #reaching the i'th seedling's location from an unsampled tree j 
    #distance units outside of the plot
    osd[which(is.na(osd))] <- 1e-100
    #Pollen expected to go from sampled adults to hypothetical outside adults:
    iopol <- iopol.func(up) ##iopol[i,j] is the total probability of pollen 
    iopol[which(is.na(iopol))] <- 1e-100
    #from the i'th sampled tree's location reaching the j'th distance 
    #unit outside of the plot
    #Expected pollen arriving at sampled trees from outside:
    op    <- out.pol.func(up) 
    op[which(is.na(op))] <- 1e-100
    #Pollen expected to go from hypothetical outer tree to outer tree:
    otop  <- otop.func(up)
    otop[which(is.na(otop))] <- 1e-100
  } # end "if first loop"
  #### propose new dispersal parameters
  #current parent estimate 
  A1 <- ped.gibbs[[d]][,1] # proposed mothers of offspring k (see noff loop below)
  A2 <- ped.gibbs[[d]][,2] # proposed fathers of offspring k (see noff loop below)
  usstar   <- tnorm(1,los,his,us,20) #pick a new seed dispersal parameter 
  upstar   <- tnorm(1,lop,hip,up,20) #pick a new pollen dispersal parameter
  dprob.s  <- dprob.func(usstar)     #seed dispersal probs
  dprob.s[which(is.na(dprob.s))] <- 1e-100
  pprob.s  <- pprob.func(upstar)     #pollen dispersal probs
  pprob.s[which(is.na(pprob.s))] <- 1e-100
  osd.s    <- out.sd.func(usstar)    #seed to plot [row] from outside mother[col]
  osd.s[which(is.na(osd.s))] <- 1e-100
  iopol.s  <- iopol.func(upstar)     #pollen to outside mother[col] from inside father[row] 
  iopol.s[which(is.na(iopol.s))] <- 1e-100
  op.s     <- out.pol.func(upstar)   #pollen to inside mother [row] from outside fathers [col] 
  op.s[which(is.na(op.s))] <- 1e-100
  otop.s   <- otop.func(upstar)      #pollen to outside mother[row] from outside fathers[col]
  otop.s[which(is.na(otop.s))] <- 1e-100
  #pollen recieved by out mother[i] from any out father:
  otopsum  <- apply(otop,1,sum); 
  otopsum.s<- apply(otop.s,1,sum)
  p        <- rep(NA,noff) 
  pstar    <- rep(NA,noff)
  #rows = recipients, col= donors
  wmat     <- matrix(w,noff,nadult,byrow=T) # w is the weights vector
  polmat   <- matrix(w,nadult,nadult,byrow=T)
  #pld is the offspring plot size, in my case == 1m^2
  plmat    <- matrix(pld,noff,nadult) #gives a matrix where each column repeats the area of the offsprings' plot sizes
  sd.pr    <- dprob*wmat*plmat 		#prob of seed dispersal from sampled adult to sampled seedling 
  sd.pr.s  <- dprob.s*wmat*plmat
  sd.pr    <- cbind(sd.pr,osd) 		#seed received from inside adults + outside adults 
  sd.pr.s  <- cbind(sd.pr.s,osd.s)
  pol.pr   <- pprob*polmat 
  pol.pr.s <- pprob.s*polmat
  pol.pr   <- cbind(pol.pr,op) 		#pollen received by adults from inside + outside adults 
  pol.pr.s <- cbind(pol.pr.s,op.s) 	#pollen received by outside adults from inside + outside adults 
  om       <- cbind(t(iopol),otop); 
  om.s     <- cbind(t(iopol.s),otop.s)
  pol.pr   <- rbind(pol.pr,om) 
  pol.pr.s <- rbind(pol.pr.s,om.s)
  #probability that current parents disperse seed and pollen given current and proposed dispersal parameters
  # Error in p[k] <- sp.pr[A1[k], A2[k]] : replacement has length zero
  print("starting offspring loop")
  for (k in 1:noff)
  {
    #ldf and ldm are seq of hypothetical father/mother distances from plot
    sp.pr   <- pol.pr*matrix(sd.pr[k,],nadult+ldm,nadult+ldf) 
    sp.pr.s <- pol.pr.s*matrix(sd.pr.s[k,],nadult+ldm,nadult+ldf)
    
    sp.pr   <- sp.pr/sum(sp.pr) 
    sp.pr.s <- sp.pr.s/sum(sp.pr.s)
    if(A1[k]<nadult+1) # if mother in stand
    { 
      if(A2[k]<nadult+1) #if both in stand
      {
        p[k]     <- sp.pr[A1[k],A2[k]]
        pstar[k] <- sp.pr.s[A1[k],A2[k]] 
      }
      if(A2[k]>nadult) #if father outside stand
      { 
        p[k]     <- sum(sp.pr[A1[k],(nadult+1):(nadult+ldf)]) 
        pstar[k] <- sum(sp.pr.s[A1[k],(nadult+1):(nadult+ldf)])
      } 
    }
    if(A1[k]>nadult) #if mother outside stand
    {  
      if(A2[k]<nadult+1) #if father in stand
      {
        p[k]     <- sum(sp.pr[(nadult+1):(nadult+ldm),A2[k]])
        pstar[k] <- sum(sp.pr.s[(nadult+1):(nadult+ldm),A2[k]]) 
      }
      if(A2[k]>nadult) #if both outside the stand
      { 
        p[k]     <- sum(sp.pr[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)]) 
        pstar[k] <- sum(sp.pr.s[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)])
      } 
    }
    #"pre-propose" parents for next loop of gibbs sampler
    #translates probabilities into a vector, upper triangle first, by column 
    theta <- c(ppmat[[k]][upper.tri(ppmat[[k]])],ppmat[[k]][lower.tri(ppmat[[k]])],ppmat[[k]][(nadult+ 1),(nadult+1)])
    y     <- rmultinom(1,1,theta) 
    ly    <- length(y)-1
    ul    <- .5*ly; 
    ll    <- ul+1
    v     <- matrix(0,(nadult+1),(nadult+1)) 
    v[upper.tri(v)]<- y[c(1:ul)]
    v[lower.tri(v)] <- y[c(ll:ly)] 
    v[(nadult+1),(nadult+1)] <- y[length(y)]
    ped.new[k,1] <- which(apply(v,1,sum)==1) 
    ped.new[k,2] <- which(apply(v,2,sum)==1)
    tnow[k]   <- p[k]
    tnow.s[k] <- pstar[k]
    if(ped.new[k,1]<nadult+1)
    { 
      if(ped.new[k,2]<nadult+1)
      {
        tnew[k]   <- sp.pr[ped.new[k,1],ped.new[k,2]]
        tnew.s[k] <- sp.pr.s[ped.new[k,1],ped.new[k,2]] 
      }
      if(ped.new[k,2]>nadult)
      { 
        tnew[k]   <- sum(sp.pr[ped.new[k,1],(nadult+1):(nadult+ldf)]) 
        tnew.s[k] <- sum(sp.pr.s[ped.new[k,1],(nadult+1):(nadult+ldf)])
      } 
    }
    if(ped.new[k,1]>nadult)
    { 
      if(ped.new[k,2]<nadult+1)
      {
        tnew[k]   <- sum(sp.pr[(nadult+1):(nadult+ldm),ped.new[k,2]])
        tnew.s[k] <- sum(sp.pr.s[(nadult+1):(nadult+ldm),ped.new[k,2]])
      }
      if(ped.new[k,2]>nadult)
      { 
        tnew[k]   <- sum(sp.pr[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)]) 
        tnew.s[k] <- sum(sp.pr.s[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)])
      } 
    }
  }#end offspring loop
  q1 <- sum(log(p))
  q2 <- sum(log(pstar))
  
  #sum over all offspring, times priors
  pnow <- q1+dnorm(up,mpp,sdpp,log=TRUE)+dnorm(us,msp,sdsp,log=TRUE)
  pnew <- q2+dnorm(upstar,mpp,sdpp,log=TRUE)+dnorm(usstar,msp,sdsp,log=TRUE)
  r  <- exp(pnew-pnow)
  z  <- runif(1,0,1) #accept or reject proposed values 
  if(z < r)
  {
    up <- upstar 
    us <- usstar 
    dprob <- dprob.s
    pprob <- pprob.s 
    osd   <- osd.s 
    iopol <- iopol.s 
    op    <- op.s
    otop  <- otop.s
    tnew  <- tnew.s
    tnow  <- tnow.s
  }
  usgibbs <- c(usgibbs,us) 
  upgibbs <- c(upgibbs,up)
  
  ####propose pedigree 
  for(k in 1:noff)
  {
    cat(sprintf("pedigree %s\n",k))
    #keep track of parameter values
    #probability that current or proposed parents could produce observed offspring genotype 
    gnow <- amat[[k]][ped.gibbs[[d]][k,1],ped.gibbs[[d]][k,2]]
    gnew <- amat[[k]][ped.new[k,1],ped.new[k,2]]
    
    #...combined with probability of dispersal given current dispersal parameters 
    pnow <- log(gnow)+log(tnow[k])
    pnew <- log(gnew)+log(tnew[k])
    r <- exp(pnew-pnow)
    z <- runif(1,0,1)
    if(z < r)
    { #accept or reject 
      ped.gibbs[[d+1]][k,1]<-ped.new[k,1] 
      ped.gibbs[[d+1]][k,2]<-ped.new[k,2]
    }
    if(z >= r)
    {
      ped.gibbs[[d+1]][k,1]<-ped.gibbs[[d]][k,1]
      ped.gibbs[[d+1]][k,2]<- ped.gibbs[[d]][k,2] 
    }
  }# end k loop over offspring 
  print(d)
}#end cycle

dir <- function(dir,name){
    return(paste(dir,name,sep=""))
}

DIR = "/home/lindb/teakettle/data/gibbs/cycle_data/"
for (obj in ls()){
    saveRDS(get(obj),dir(DIR,sprintf("%s.RDS",obj)))
}


'''
cDIR = '/home/lindb/teakettle/data/gibbs/cycle_data'
if not op.exists(cDIR):
    os.makedirs(cDIR)
DIR = '/home/lindb/teakettle/data/gibbs'
if not op.exists(DIR):
    os.makedirs(DIR)
filE = op.join(DIR,'gibbs.R')
with open(filE,'w') as o:
    o.write("%s" % text)

In [ ]:
text = '''
source('/home/lindb/imports.R')

#load dependencies
dname = '/home/lindb/teakettle/data/moran/dependencies/'
files = c()
for (f in list.files(dname)){
    if (endsWith(f,'RDS'))
    {
        sprintf ("loading %%s file",f)
        fname = paste(dname,f,sep="")
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(fname))    
    }
}

pld <- as.numeric(pld)

amat <- readRDS('/home/lindb/teakettle/data/small/combo/amat_upper_tri_summed.RDS')
###### just this time - fix when I have lower tri
#for (k in 1:noff){
#  amat[[k]][lower.tri(amat[[k]])] <- t(amat[[k]])[lower.tri(amat[[k]])]
#}
lapply(amat,function(x) {x[lower.tri(x)]<-t(x)[lower.tri(x)];x})

###############################################################
################## Step 5: the Gibbs Sampler ##################
###############################################################
#number of Gibbs steps
ncyc <- 50000
#Starting values for seed and pollen parameters
#usgibbs <- us <- 150
#upgibbs <- up <- 500 
us <- 150
up <- 500 

###Which parent pairs will you consider? 
##Can consider all with a >0 probability of producing observed 
#offspring genotype (as below) or can set more stringent standard.
ppmat <- numeric(0)
for(k in 1:noff)
{
	print(k)
	b <- matrix(0,(nadult+1),(nadult+1))
	b[which(amat[[k]] > 0)] <- 1 #way faster than this loop below
#	for(i in 1:(nadult+1))
#	{
#		for(j in 1:(nadult+1))
#		{ 
#			##amat_k[i,j] represents the probability of obtaining the observed genotype of 
#			#offspring k given the pedigree, P_k = (i,j), relative to all possible parent combinations
#			if(amat[[k]][i,j]>0)
#			{
#				b[i,j]<-1
#			}
#		}
#	}
	#ppmat is a list of matrices of len(noff), each element a matrix of 0s and 1s. 1s = Amat_k[i,j]>0
	ppmat <- append(ppmat,list(b)) 
}

#Initial parentage values – start with parents that have high probability of producing observed offspring genotypes.
Ped.eg <- matrix(0,noff,2)
print("IDing first parents")
for (k in 1:noff)
{ #translates probabilities into a vector, upper triangle first, by column
	theta <- c(amat[[k]][upper.tri(amat[[k]])],amat[[k]][lower.tri(amat[[k]])],amat[[k]][(nadult+1),(nadult+1)])
	y     <- rmultinom(1,1,theta) 
	# ly = twice the total number of parent pair combinations where i!=j (plus one (amat_k[bottom right cell]), minus one)
	ly    <- length(y)-1 
	ul    <- .5*ly; #total number of parent pair combinations where i!=j
	ll    <- ul+1 #total number of parent pair combinations where i!=j plus one
	v     <- matrix(0,(nadult+1),(nadult+1)) 
	v[upper.tri(v)] <- y[c(1:ul)] #fills in the first half of multinomially distributed random number (parent pair combos)
	v[lower.tri(v)] <- y[c(ll:ly)] #fills in the second half of multinomially distributed random number (parent pair combos)
	#next add in the multinomially distributed random number corresponding to two parents outside the plot
	v[(nadult+1),(nadult+1)] <- y[length(y)] 
	Ped.eg[k,1] <- which(apply(v,1,sum)==1) #figure out the i'th coordinate(s) for the parent pair(s) within the matrix
	Ped.eg[k,2] <- which(apply(v,2,sum)==1) #figure out the j'th coordinate(s) for the parent pair(s) within the matrix
}

## to hold parentage estimates
#ped.gibbs <- numeric(0)
#ped.gibbs <- append(ped.gibbs,list(matrix(0,noff,2)))
#ped.gibbs[[1]] <- Ped.eg
## new
ped.gibbs <- Ped.eg


#Begin Gibbs Sampler:
#set all NA probs to ~0 -
print("starting Gibbs")
for (d in 1:ncyc)
{
  print(d)
  #ped.gibbs <- append(ped.gibbs,list(matrix(0,noff,2))) #don't think I need this
  ped.gibbsnext <- matrix(0,noff,2)
  ped.new   <- matrix(0,noff,2)
  tnow      <- rep(0,noff); 
  tnow.s    <- rep(0,noff) 
  tnew      <- rep(0,noff); 
  tnew.s    <- rep(0,noff)
  if(d==1)
  { #just the first time
    #Probability of seed dispersal from sampled adult to location of each seedling:
    dprob <- dprob.func(us) #seedling shape parameter, us, defined around line 1042
    dprob[which(is.na(dprob))] <- 1e-100
    #Probability of pollen dispersal between sampled adults:
    pprob <- pprob.func(up) #pollen shape parameter, up, defined around line 1043
    pprob[which(is.na(pprob))] <- 1e-100 
    #Expected seed arriving at seedling plots from outside plot:
    osd   <- out.sd.func(us) ##osd[i,j] is the total probability of a seed 
    #reaching the i'th seedling's location from an unsampled tree j 
    #distance units outside of the plot
    osd[which(is.na(osd))] <- 1e-100
    #Pollen expected to go from sampled adults to hypothetical outside adults:
    iopol <- iopol.func(up) ##iopol[i,j] is the total probability of pollen 
    iopol[which(is.na(iopol))] <- 1e-100
    #from the i'th sampled tree's location reaching the j'th distance 
    #unit outside of the plot
    #Expected pollen arriving at sampled trees from outside:
    op    <- out.pol.func(up) 
    op[which(is.na(op))] <- 1e-100
    #Pollen expected to go from hypothetical outer tree to outer tree:
    otop  <- otop.func(up)
    otop[which(is.na(otop))] <- 1e-100
  } # end "if first loop"
  #### propose new dispersal parameters
  #current parent estimate 
  #A1 <- ped.gibbs[[d]][,1] # proposed mothers of offspring k (see noff loop below)
  #A2 <- ped.gibbs[[d]][,2] # proposed fathers of offspring k (see noff loop below)
  #faster current parent  estimate
  A1 <- ped.gibbs[,1] # proposed mothers of offspring k (see noff loop below)
  A2 <- ped.gibbs[,2] # proposed fathers of offspring k (see noff loop below)
  usstar   <- tnorm(1,los,his,us,20) #pick a new seed dispersal parameter 
  upstar   <- tnorm(1,lop,hip,up,20) #pick a new pollen dispersal parameter
  dprob.s  <- dprob.func(usstar)     #seed dispersal probs
  dprob.s[which(is.na(dprob.s))] <- 1e-100
  pprob.s  <- pprob.func(upstar)     #pollen dispersal probs
  pprob.s[which(is.na(pprob.s))] <- 1e-100
  osd.s    <- out.sd.func(usstar)    #seed to plot [row] from outside mother[col]
  osd.s[which(is.na(osd.s))] <- 1e-100
  iopol.s  <- iopol.func(upstar)     #pollen to outside mother[col] from inside father[row] 
  iopol.s[which(is.na(iopol.s))] <- 1e-100
  op.s     <- out.pol.func(upstar)   #pollen to inside mother [row] from outside fathers [col] 
  op.s[which(is.na(op.s))] <- 1e-100
  otop.s   <- otop.func(upstar)      #pollen to outside mother[row] from outside fathers[col]
  otop.s[which(is.na(otop.s))] <- 1e-100
  #pollen recieved by out mother[i] from any out father:
  otopsum  <- apply(otop,1,sum); 
  otopsum.s<- apply(otop.s,1,sum)
  p        <- rep(NA,noff) 
  pstar    <- rep(NA,noff)
  #rows = recipients, col= donors
  wmat     <- matrix(w,noff,nadult,byrow=T) # w is the weights vector
  polmat   <- matrix(w,nadult,nadult,byrow=T)
  #pld is the offspring plot size, in my case == 1m^2
  plmat    <- matrix(pld,noff,nadult) #gives a matrix where each column repeats the area of the offsprings' plot sizes
  sd.pr    <- dprob*wmat*plmat 		#prob of seed dispersal from sampled adult to sampled seedling 
  sd.pr.s  <- dprob.s*wmat*plmat
  sd.pr    <- cbind(sd.pr,osd) 		#seed received from inside adults + outside adults 
  sd.pr.s  <- cbind(sd.pr.s,osd.s)
  pol.pr   <- pprob*polmat 
  pol.pr.s <- pprob.s*polmat
  pol.pr   <- cbind(pol.pr,op) 		#pollen received by adults from inside + outside adults 
  pol.pr.s <- cbind(pol.pr.s,op.s) 	#pollen received by outside adults from inside + outside adults 
  om       <- cbind(t(iopol),otop); 
  om.s     <- cbind(t(iopol.s),otop.s)
  pol.pr   <- rbind(pol.pr,om) 
  pol.pr.s <- rbind(pol.pr.s,om.s)
  #probability that current parents disperse seed and pollen given current and proposed dispersal parameters
  # Error in p[k] <- sp.pr[A1[k], A2[k]] : replacement has length zero
  print("starting offspring loop")
  for (k in 1:noff)
  {
    #ldf and ldm are seq of hypothetical father/mother distances from plot
    sp.pr   <- pol.pr*matrix(sd.pr[k,],nadult+ldm,nadult+ldf) 
    sp.pr.s <- pol.pr.s*matrix(sd.pr.s[k,],nadult+ldm,nadult+ldf)
    
    sp.pr   <- sp.pr/sum(sp.pr) 
    sp.pr.s <- sp.pr.s/sum(sp.pr.s)
    if(A1[k]<nadult+1) # if mother in stand
    { 
      if(A2[k]<nadult+1) #if both in stand
      {
        p[k]     <- sp.pr[A1[k],A2[k]]
        pstar[k] <- sp.pr.s[A1[k],A2[k]] 
      }
      if(A2[k]>nadult) #if father outside stand
      { 
        p[k]     <- sum(sp.pr[A1[k],(nadult+1):(nadult+ldf)]) 
        pstar[k] <- sum(sp.pr.s[A1[k],(nadult+1):(nadult+ldf)])
      } 
    }
    if(A1[k]>nadult) #if mother outside stand
    {  
      if(A2[k]<nadult+1) #if father in stand
      {
        p[k]     <- sum(sp.pr[(nadult+1):(nadult+ldm),A2[k]])
        pstar[k] <- sum(sp.pr.s[(nadult+1):(nadult+ldm),A2[k]]) 
      }
      if(A2[k]>nadult) #if both outside the stand
      { 
        p[k]     <- sum(sp.pr[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)]) 
        pstar[k] <- sum(sp.pr.s[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)])
      } 
    }
    #"pre-propose" parents for next loop of gibbs sampler
    #translates probabilities into a vector, upper triangle first, by column 
    theta <- c(ppmat[[k]][upper.tri(ppmat[[k]])],ppmat[[k]][lower.tri(ppmat[[k]])],ppmat[[k]][(nadult+ 1),(nadult+1)])
    y     <- rmultinom(1,1,theta) 
    ly    <- length(y)-1
    ul    <- .5*ly; 
    ll    <- ul+1
    v     <- matrix(0,(nadult+1),(nadult+1)) 
    v[upper.tri(v)]<- y[c(1:ul)]
    v[lower.tri(v)] <- y[c(ll:ly)] 
    v[(nadult+1),(nadult+1)] <- y[length(y)]
    ped.new[k,1] <- which(apply(v,1,sum)==1) 
    ped.new[k,2] <- which(apply(v,2,sum)==1)
    tnow[k]   <- p[k]
    tnow.s[k] <- pstar[k]
    if(ped.new[k,1]<nadult+1)
    { 
      if(ped.new[k,2]<nadult+1)
      {
        tnew[k]   <- sp.pr[ped.new[k,1],ped.new[k,2]]
        tnew.s[k] <- sp.pr.s[ped.new[k,1],ped.new[k,2]] 
      }
      if(ped.new[k,2]>nadult)
      { 
        tnew[k]   <- sum(sp.pr[ped.new[k,1],(nadult+1):(nadult+ldf)]) 
        tnew.s[k] <- sum(sp.pr.s[ped.new[k,1],(nadult+1):(nadult+ldf)])
      } 
    }
    if(ped.new[k,1]>nadult)
    { 
      if(ped.new[k,2]<nadult+1)
      {
        tnew[k]   <- sum(sp.pr[(nadult+1):(nadult+ldm),ped.new[k,2]])
        tnew.s[k] <- sum(sp.pr.s[(nadult+1):(nadult+ldm),ped.new[k,2]])
      }
      if(ped.new[k,2]>nadult)
      { 
        tnew[k]   <- sum(sp.pr[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)]) 
        tnew.s[k] <- sum(sp.pr.s[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)])
      } 
    }
  }#end offspring loop
  q1 <- sum(log(p))
  q2 <- sum(log(pstar))
  
  #sum over all offspring, times priors
  pnow <- q1+dnorm(up,mpp,sdpp,log=TRUE)+dnorm(us,msp,sdsp,log=TRUE)
  pnew <- q2+dnorm(upstar,mpp,sdpp,log=TRUE)+dnorm(usstar,msp,sdsp,log=TRUE)
  r  <- exp(pnew-pnow)
  z  <- runif(1,0,1) #accept or reject proposed values 
  if(z < r)
  {
    up <- upstar 
    us <- usstar 
    dprob <- dprob.s
    pprob <- pprob.s 
    osd   <- osd.s 
    iopol <- iopol.s 
    op    <- op.s
    otop  <- otop.s
    tnew  <- tnew.s
    tnow  <- tnow.s
  }
  x <- d
  while (nchar(x)<5)
  {
    x = sprintf('0%s',x)
  }
  #usgibbs <- c(usgibbs,us) #### save
  saveRDS(us,sprintf('/home/lindb/teakettle/data/gibbs/cycle_data/usgibbs_%s',x))
  #upgibbs <- c(upgibbs,up) #### save
  saveRDS(up,sprintf('/home/lindb/teakettle/data/gibbs/cycle_data/upgibbs_%s',x))
  
  ####propose pedigree 
  for(k in 1:noff)
  {
    cat(sprintf("pedigree %s\n",k))
    #keep track of parameter values
    #probability that current or proposed parents could produce observed offspring genotype 
    #gnow <- amat[[k]][ped.gibbs[[d]][k,1],ped.gibbs[[d]][k,2]]
    #gnew <- amat[[k]][ped.new[k,1],ped.new[k,2]]
    #faster probability that current or proposed parents could produce observed offspring genotype 
    gnow <- amat[[k]][ped.gibbs[k,1],ped.gibbs[k,2]]
    gnew <- amat[[k]][ped.new[k,1],ped.new[k,2]]
    
    #...combined with probability of dispersal given current dispersal parameters 
    pnow <- log(gnow)+log(tnow[k])
    pnew <- log(gnew)+log(tnew[k])
    r <- exp(pnew-pnow)
    z <- runif(1,0,1)
    if(z < r)
    { #accept or reject 
      #ped.gibbs[[d+1]][k,1]<-ped.new[k,1] 
      #ped.gibbs[[d+1]][k,2]<-ped.new[k,2]
      #faster accept or reject
      ped.gibbsnext[k,1]<-ped.new[k,1] 
      ped.gibbsnext[k,2]<-ped.new[k,2]
    }
    if(z >= r)
    {
      #ped.gibbs[[d+1]][k,1]<-ped.gibbs[[d]][k,1]
      #ped.gibbs[[d+1]][k,2]<- ped.gibbs[[d]][k,2] 
      #faster
      ped.gibbsnext[k,1]<-ped.gibbs[k,1]
      ped.gibbsnext[k,2]<- ped.gibbs[k,2] 
    }
  }# end k loop over offspring 
  x <- d
  while (nchar(x)<5)
  {
    x = sprintf('0%s',x)
  }
  saveRDS(ped.gibbs,sprintf('/home/lindb/teakettle/data/gibbs/cycle_data/ped.gibbs_%s',x))
  ped.gibbs <- ped.gibbsnext
  print(d)
}#end cycle

dir <- function(dir,name){
    return(paste(dir,name,sep=""))
}

DIR = "/home/lindb/teakettle/data/gibbs/cycle_data/"
for (obj in ls()){
    saveRDS(get(obj),dir(DIR,sprintf("%s.RDS",obj)))
}

'''
cDIR = '/home/lindb/teakettle/data/gibbs/cycle_data'
if not op.exists(cDIR):
    os.makedirs(cDIR)
DIR = '/home/lindb/teakettle/data/gibbs'
if not op.exists(DIR):
    os.makedirs(DIR)
filE = op.join(DIR,'gibbs_faster.R')
with open(filE,'w') as o:
    o.write("%s" % text)

In [ ]:
# started on tmux godel30 at about 1:34pm june 14

# optimize parallelization of Gibbs sampler

In [ ]:
DIR = '/home/lindb/teakettle/data/moran/dependencies/optimize/'
if not op.exists(DIR):
    os.makedirs(DIR)

##### run the following in R to get the stuff inbetween end of amat loop and Gibbs sampler, then write script for sampler

```R
source('/home/lindb/imports.R')

#load dependencies
dname = '/home/lindb/teakettle/data/moran/dependencies/'
files = c()
for (f in sort(list.files(dname))){
    if (endsWith(f,'RDS'))
    {
        cat(sprintf ("loading %%s file",f))
        fname = paste(dname,f,sep="")
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(fname))    
    }
}


pld <- as.numeric(pld)
saveRDS(pld,'/home/lindb/teakettle/data/moran/dependencies/pld.RDS')

amat <- readRDS('/home/lindb/teakettle/data/small/combo/amat_upper_tri_summed.RDS')
amat <- lapply(amat,function(x) {x[lower.tri(x)]<-t(x)[lower.tri(x)];x})
saveRDS(amat,'/home/lindb/teakettle/data/small/combo/amat_upper_tri_summed_reflected.RDS')

###############################################################
################## Step 5: the Gibbs Sampler ##################
###############################################################
#number of Gibbs steps
ncyc <- 50000
saveRDS(ncyc,'/home/lindb/teakettle/data/moran/dependencies/ncyc.RDS')
#Starting values for seed and pollen parameters
#usgibbs <- us <- 150
#upgibbs <- up <- 500 
us <- 150
up <- 500 
saveRDS(us,'/home/lindb/teakettle/data/moran/dependencies/ped.gibbs.RDS')
saveRDS(up,'/home/lindb/teakettle/data/moran/dependencies/up.RDS')

###Which parent pairs will you consider? 
##Can consider all with a >0 probability of producing observed 
#offspring genotype (as below) or can set more stringent standard.
ppmat <- numeric(0)
for(k in 1:noff)
{
	print(k)
	b <- matrix(0,(nadult+1),(nadult+1))
	b[which(amat[[k]] > 0)] <- 1 #way faster than this loop below
#	for(i in 1:(nadult+1))
#	{
#		for(j in 1:(nadult+1))
#		{ 
#			##amat_k[i,j] represents the probability of obtaining the observed genotype of 
#			#offspring k given the pedigree, P_k = (i,j), relative to all possible parent combinations
#			if(amat[[k]][i,j]>0)
#			{
#				b[i,j]<-1
#			}
#		}
#	}
	#ppmat is a list of matrices of len(noff), each element a matrix of 0s and 1s. 1s = Amat_k[i,j]>0
	ppmat <- append(ppmat,list(b)) 
}
saveRDS(ppmat,'/home/lindb/teakettle/data/moran/dependencies/ppmat.RDS')

#Initial parentage values – start with parents that highest probability of producing observed offspring genotypes.
Ped.eg <- matrix(0,noff,2)
print("IDing first parents")
for (k in 1:noff)
{ #translates probabilities into a vector, upper triangle first, by column
	theta <- c(amat[[k]][upper.tri(amat[[k]])],amat[[k]][lower.tri(amat[[k]])],amat[[k]][(nadult+1),(nadult+1)])
	y     <- rmultinom(1,1,theta) 
	# ly = twice the total number of parent pair combinations where i!=j (plus one (amat_k[bottom right cell]), minus one)
	ly    <- length(y)-1 
	ul    <- .5*ly; #total number of parent pair combinations where i!=j
	ll    <- ul+1 #total number of parent pair combinations where i!=j plus one
	v     <- matrix(0,(nadult+1),(nadult+1)) 
	v[upper.tri(v)] <- y[c(1:ul)] #fills in the first half of multinomially distributed random number (parent pair combos)
	v[lower.tri(v)] <- y[c(ll:ly)] #fills in the second half of multinomially distributed random number (parent pair combos)
	#next add in the multinomially distributed random number corresponding to two parents outside the plot
	v[(nadult+1),(nadult+1)] <- y[length(y)] 
	Ped.eg[k,1] <- which(apply(v,1,sum)==1) #figure out the i'th coordinate(s) for the parent pair(s) within the matrix
	Ped.eg[k,2] <- which(apply(v,2,sum)==1) #figure out the j'th coordinate(s) for the parent pair(s) within the matrix
}
#saveRDS(Ped.eg,'/home/lindb/teakettle/data/moran/dependencies/Ped.eg.RDS')


## to hold parentage estimates
#ped.gibbs <- numeric(0)
#ped.gibbs <- append(ped.gibbs,list(matrix(0,noff,2)))
#ped.gibbs[[1]] <- Ped.eg
## new
#do this, but save a little time: ped.gibbs <- Ped.eg
saveRDS(Ped.eg,'/home/lindb/teakettle/data/moran/dependencies/ped.gibbs.RDS')

#special function to speed up gibbs sampler
dostuff <- function(k){
	#ldf and ldm are seq of hypothetical father/mother distances from plot
	#pol.pr is nrow=ncol=nadult, where rows are the recipients of pollen, cols the donor, elements the probability
	#sd.pr is nrow=noffspring ncol=nadult, element is probability of seed dispersal to offspring from adult
	#nadult = 558, noff=718
	sp.pr   <- pol.pr*matrix(sd.pr[k,],nadult+ldm,nadult+ldf)        #dim(sp.pr)   == 568 x 568
	sp.pr.s <- pol.pr.s*matrix(sd.pr.s[k,],nadult+ldm,nadult+ldf)    #dim(sp.pr.s) == 568 x 568

	sp.pr   <- sp.pr/sum(sp.pr) 
	sp.pr.s <- sp.pr.s/sum(sp.pr.s)
	if(A1[k]<nadult+1) # if mother in stand
	{ 
	  if(A2[k]<nadult+1) #if both in stand
	  {
	    p     <- sp.pr[A1[k],A2[k]]
	    pstar <- sp.pr.s[A1[k],A2[k]] 
	  }
	  if(A2[k]>nadult) #if father outside stand
	  { 
	    p     <- sum(sp.pr[A1[k],(nadult+1):(nadult+ldf)]) 
	    pstar <- sum(sp.pr.s[A1[k],(nadult+1):(nadult+ldf)])
	  } 
	}
	if(A1[k]>nadult) #if mother outside stand
	{  
	  if(A2[k]<nadult+1) #if father in stand
	  {
	    p     <- sum(sp.pr[(nadult+1):(nadult+ldm),A2[k]])
	    pstar <- sum(sp.pr.s[(nadult+1):(nadult+ldm),A2[k]]) 
	  }
	  if(A2[k]>nadult) #if both outside the stand
	  { 
	    p     <- sum(sp.pr[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)]) 
	    pstar <- sum(sp.pr.s[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)])
	  } 
	}
	#"pre-propose" parents for next loop of gibbs sampler
	#translates probabilities into a vector, upper triangle first, by column 
	theta <- c(ppmat[[k]][upper.tri(ppmat[[k]])],ppmat[[k]][lower.tri(ppmat[[k]])],ppmat[[k]][(nadult+ 1),(nadult+1)])
	y     <- rmultinom(1,1,theta) 
	ly    <- length(y)-1
	ul    <- .5*ly; 
	ll    <- ul+1
	v     <- matrix(0,(nadult+1),(nadult+1)) 
	v[upper.tri(v)]<- y[c(1:ul)]
	v[lower.tri(v)] <- y[c(ll:ly)] 
	v[(nadult+1),(nadult+1)] <- y[length(y)]
	# ped.new[k,1] <- which(apply(v,1,sum)==1) 
	# ped.new[k,2] <- which(apply(v,2,sum)==1)
	ped.new1 <- which(apply(v,1,sum)==1) 
	ped.new2 <- which(apply(v,2,sum)==1)
	tnow   <- p
	tnow.s <- pstar
	if(ped.new1<nadult+1)
	{ 
	  if(ped.new2<nadult+1)
	  {
	    tnew   <- sp.pr[ped.new1,ped.new2]
	    tnew.s <- sp.pr.s[ped.new1,ped.new2]
	  }
	  if(ped.new2>nadult)
	  { 
	    tnew   <- sum(sp.pr[ped.new1,(nadult+1):(nadult+ldf)]) 
	    tnew.s <- sum(sp.pr.s[ped.new1,(nadult+1):(nadult+ldf)])
	  } 
	}
	if(ped.new1>nadult)
	{ 
	  if(ped.new2<nadult+1)
	  {
	    tnew   <- sum(sp.pr[(nadult+1):(nadult+ldm),ped.new2])
	    tnew.s <- sum(sp.pr.s[(nadult+1):(nadult+ldm),ped.new2])
	  }
	  if(ped.new2>nadult)
	  { 
	    tnew   <- sum(sp.pr[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)]) 
	    tnew.s <- sum(sp.pr.s[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)])
	  } 
	}
	return(list(k,
	p,
	pstar,
	ped.new1,
	ped.new2,
	tnow,
	tnow.s,
	tnew,
	tnew.s
	))
}
saveRDS(dostuff,'/home/lindb/teakettle/data/moran/dependencies/dostuff.RDS')

```

In [ ]:
#how many engines to try?
engines = ['04',
           '08',
           '12',
           '16',
           '20',
           '24',
           '30',
           '50',
           '100',
           '150'
      ]

In [ ]:
DIR = '/home/lindb/teakettle/data/gibbs/optimize/scripts'
if not op.exists(DIR):
    os.makedirs(DIR)

for eng in engines:
    text = '''
source("/home/lindb/imports.R")

eng <- as.numeric(%s)

#load dependencies
dname = "/home/lindb/teakettle/data/moran/dependencies2/"
files = c()
for (f in sort(list.files(dname))){
    if (endsWith(f,"RDS"))
    {
        cat(sprintf ("loading %%s file\\n",f))
        fname = paste(dname,f,sep="")
        obj = strsplit(f,split=".RDS")[[1]]
        assign(obj,readRDS(fname))    
    }
}

amat <- readRDS("/home/lindb/teakettle/data/small/combo/amat_upper_tri_summed_reflected.RDS")

library(snow)
cl = makeSOCKcluster(eng)
exports = c("ppmat",
            "ldf",
            "ldm",
            "nadult"
            )
clusterExport(cl,exports)

ncyc <- 5
loop <- as.array(1:noff)
#Begin Gibbs Sampler:
#set all NA probs to ~0 -
print("starting Gibbs")
for(d in 1:ncyc)
{
    start <- Sys.time()
    print(d)

    ped.gibbsnext <- matrix(0,noff,2)
    ped.new   <- matrix(0,noff,2)
    tnow      <- rep(0,noff); 
    tnow.s    <- rep(0,noff) 
    tnew      <- rep(0,noff); 
    tnew.s    <- rep(0,noff)
    if(d==1)
    { #just the first time
        #Probability of seed dispersal from sampled adult to location of each seedling:
        dprob <- dprob.func(us) #seedling shape parameter, us, defined around line 1042
        dprob[which(is.na(dprob))] <- 1e-100 #these are because I accidentally included 4 trees without coordinates, no going back until after Evoln
        #Probability of pollen dispersal between sampled adults:
        pprob <- pprob.func(up) #pollen shape parameter, up, defined around line 1043
        pprob[which(is.na(pprob))] <- 1e-100 
        #Expected seed arriving at seedling plots from outside plot:
        osd   <- out.sd.func(us) ##osd[i,j] is the total probability of a seed 
        #reaching the ith seedlings location from an unsampled tree j 
        #distance units outside of the plot
        osd[which(is.na(osd))] <- 1e-100
        #Pollen expected to go from sampled adults to hypothetical outside adults:
        iopol <- iopol.func(up) ##iopol[i,j] is the total probability of pollen 
        iopol[which(is.na(iopol))] <- 1e-100
        #from the ith sampled trees location reaching the jth distance 
        #unit outside of the plot
        #Expected pollen arriving at sampled trees from outside:
        op    <- out.pol.func(up) 
        op[which(is.na(op))] <- 1e-100
        #Pollen expected to go from hypothetical outer tree to outer tree:
        otop  <- otop.func(up)
        otop[which(is.na(otop))] <- 1e-100
        } # end "if first loop"
    #### propose new dispersal parameters
    #current parent estimate 
    #A1 <- ped.gibbs[[d]][,1] # proposed mothers of offspring k (see noff loop below)
    #A2 <- ped.gibbs[[d]][,2] # proposed fathers of offspring k (see noff loop below)
    #faster current parent  estimate
    A1 <- ped.gibbs[,1] # proposed mothers of offspring k (see noff loop below)
    A2 <- ped.gibbs[,2] # proposed fathers of offspring k (see noff loop below)
    usstar   <- tnorm(1,los,his,us,20) #pick a new seed dispersal parameter 
    upstar   <- tnorm(1,lop,hip,up,20) #pick a new pollen dispersal parameter
    dprob.s  <- dprob.func(usstar)     #seed dispersal probs
    dprob.s[which(is.na(dprob.s))] <- 1e-100
    pprob.s  <- pprob.func(upstar)     #pollen dispersal probs
    pprob.s[which(is.na(pprob.s))] <- 1e-100
    osd.s    <- out.sd.func(usstar)    #seed to plot [row] from outside mother[col]
    osd.s[which(is.na(osd.s))]     <- 1e-100
    iopol.s  <- iopol.func(upstar)     #pollen to outside mother[col] from inside father[row] 
    iopol.s[which(is.na(iopol.s))] <- 1e-100
    op.s     <- out.pol.func(upstar)   #pollen to inside mother [row] from outside fathers [col] 
    op.s[which(is.na(op.s))]       <- 1e-100
    otop.s   <- otop.func(upstar)      #pollen to outside mother[row] from outside fathers[col]
    otop.s[which(is.na(otop.s))]   <- 1e-100
    #pollen recieved by out mother[i] from any out father:
    otopsum  <- apply(otop,1,sum); 
    otopsum.s<- apply(otop.s,1,sum)
    # p        <- rep(NA,noff) 
    # pstar    <- rep(NA,noff)
    #rows = recipients, col= donors
    wmat     <- matrix(w,noff,nadult,byrow=T) # w is the weights vector
    polmat   <- matrix(w,nadult,nadult,byrow=T)
    #pld is the offspring plot size, in my case == 1m^2
    plmat    <- matrix(pld,noff,nadult) #gives a matrix where each column repeats the area of the offsprings plot sizes
    sd.pr    <- dprob*wmat*plmat 		#prob of seed dispersal from sampled adult to sampled seedling 
    sd.pr.s  <- dprob.s*wmat*plmat
    sd.pr    <- cbind(sd.pr,osd) 		#seed received from inside adults + outside adults 
    sd.pr.s  <- cbind(sd.pr.s,osd.s)
    pol.pr   <- pprob*polmat 
    pol.pr.s <- pprob.s*polmat
    pol.pr   <- cbind(pol.pr,op) 		#pollen received by adults from inside + outside adults 
    pol.pr.s <- cbind(pol.pr.s,op.s) 	#pollen received by outside adults from inside + outside adults 
    om       <- cbind(t(iopol),otop); 
    om.s     <- cbind(t(iopol.s),otop.s)
    pol.pr   <- rbind(pol.pr,om) 
    pol.pr.s <- rbind(pol.pr.s,om.s) 

    ### speed up the old for-loop
    exports = c("A1",
                "A2",
                "pol.pr",
                "pol.pr.s",
                "sd.pr",
                "sd.pr.s")
    clusterExport(cl,exports)
    out <- parApply(cl,loop,1,dostuff)
    p <- pstar <- tnow <- tnow.s <- tnew <- tnew.s <- pednew1 <- pednew2 <- rep(NA,noff)
    for(k in 1:noff){
        p[k]        <- out[[k]][[2]]
        pstar[k]    <- out[[k]][[3]]
        pednew1[k]	<- out[[k]][[4]]
        pednew2[k]	<- out[[k]][[5]]
        tnow[k]		<- out[[k]][[6]]
        tnow.s[k]	<- out[[k]][[7]]
        tnew[k] 	<- out[[k]][[8]]
        tnew.s[k]	<- out[[k]][[9]]
    }
    ped.new <- cbind(pednew1,pednew2)
    ### end speed up of old loop


    q1 <- sum(log(p))
    q2 <- sum(log(pstar))

    #sum over all offspring, times priors
    pnow <- q1+dnorm(up,mpp,sdpp,log=TRUE)+dnorm(us,msp,sdsp,log=TRUE)
    pnew <- q2+dnorm(upstar,mpp,sdpp,log=TRUE)+dnorm(usstar,msp,sdsp,log=TRUE)
    r    <- exp(pnew-pnow)
    z    <- runif(1,0,1) #accept or reject proposed values 
    if(z < r)
    {
        up    <- upstar 
        us    <- usstar 
        dprob <- dprob.s
        pprob <- pprob.s 
        osd   <- osd.s 
        iopol <- iopol.s 
        op    <- op.s
        otop  <- otop.s
        tnew  <- tnew.s
        tnow  <- tnow.s
    }
    x <- d
    while (nchar(x)<5)
    {
        x = sprintf("0%%s",x)
    }
    ###usgibbs <- c(usgibbs,us) #### save
    #saveRDS(us,sprintf("%%s/dependencies/usgibbs_%%s",DIR,x))
    ###upgibbs <- c(upgibbs,up) #### save
    #saveRDS(up,sprintf("%%s/dependencies/upgibbs_%%s",DIR,x))

    ####propose pedigree 
    for(k in 1:noff) #super fast, not worth multiprocessing
    {
        #cat(sprintf("pedigree %%s\\n",k))
        #keep track of parameter values
        #probability that current or proposed parents could produce observed offspring genotype 
        #gnow <- amat[[k]][ped.gibbs[[d]][k,1],ped.gibbs[[d]][k,2]]
        #gnew <- amat[[k]][ped.new[k,1],ped.new[k,2]]
        #faster probability that current or proposed parents could produce observed offspring genotype 
        gnow <- amat[[k]][ped.gibbs[k,1],ped.gibbs[k,2]]
        gnew <- amat[[k]][ped.new[k,1],ped.new[k,2]]

        #...combined with probability of dispersal given current dispersal parameters 
        pnow <- log(gnow)+log(tnow[k])
        pnew <- log(gnew)+log(tnew[k])
        r <- exp(pnew-pnow)
        z <- runif(1,0,1)
        if(z < r)
        {#accept or reject 
            #ped.gibbs[[d+1]][k,1]<-ped.new[k,1] 
            #ped.gibbs[[d+1]][k,2]<-ped.new[k,2]
            #faster accept or reject
            ped.gibbsnext[k,1]<-ped.new[k,1] 
            ped.gibbsnext[k,2]<-ped.new[k,2]
        }
        if(z >= r)
        {
            #ped.gibbs[[d+1]][k,1]<-ped.gibbs[[d]][k,1]
            #ped.gibbs[[d+1]][k,2]<- ped.gibbs[[d]][k,2] 
            #faster
            ped.gibbsnext[k,1]<-ped.gibbs[k,1]
            ped.gibbsnext[k,2]<- ped.gibbs[k,2] 
        }
    }# end k loop over offspring 
    x <- d
    while (nchar(x)<5)
    {
        x = sprintf("0%%s",x)
    }
    #saveRDS(ped.gibbs,sprintf("/home/lindb/teakettle/data/gibbs/cycle_data/ped.gibbs_%%s.RDS",x))
    ped.gibbs <- ped.gibbsnext
    end <- Sys.time()
    print(d)
    print(end-start)
}#end cycle

''' % (eng)
    filE = '/home/lindb/teakettle/data/gibbs/optimize/scripts/eng%s.R' % str(eng).zfill(2)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [ ]:
DIR = '/home/lindb/teakettle/data/gibbs/optimize/scripts/'
files = fs(DIR)
for f in files:
    num = op.basename(f).split('eng')[1].split(".R")[0]
    print num
    text = '''#!/bin/bash
#$ -N eng%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd
#$ -pe smp_2 %s

R --no-save < %s
''' % (num,
       num,
       f
      )
    filE= op.join(DIR,'eng%s.sh' % num)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [ ]:
op.basename(f).split('eng')

# seems that between 30-50 engines work fastest